In [1]:
# dataload

In [2]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

In [ ]:
class LoadDataset(Dataset):

    def __init__(self, path, img_size=416):
        with open(path, 'r') as f:
            self.images = f.read().splitlines()
            self.images = list(filter(lambda x: len(x) > 0, self.images))
        assert len(self.images) > 0, 'No images found in {}'.format(path)
        
        self.img_size = img_size
        self.labels = [x.replace('images', 'labels').replace('.bmp','.txt').replace('.jpg','.txt').replace('.png','txt') for x in self.images]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_pth = self.images[idx]
        label_pth = self.labels[idx]
        img = cv2.imread(img_pth) # BGR
        assert img is not None, 'File Not Found: {}'.format(img_pth)
        
        h, w, _ = img.shape
        img, ratio, padw, padh = letterbox(img, height=self.img_size)
        
        # load label
        labels = []
        with open(label_pth, 'r') as f:
            lines = f.read().splitlines()
        assert lines is not None, 'No annotations in: {}'.format(label_pth)
        
        x = np.array([x.split() for x in lines], dtype=np.float32)
        if x.size > 0:
            # shit xywh to pixel xyxy of padded.
            labels = x.copy()
            labels[:, 1] = ratio * w * (x[:, 1] - x[:, 3] / 2) + padw
            labels[:, 2] = ratio * h * (x[:, 2] - x[:, 4] / 2) + padh
            labels[:, 3] = ratio * w * (x[:, 1] + x[:, 3] / 2) + padw
            labels[:, 4] = ratio * h * (x[:, 2] + x[:, 4] / 2) + padh
            
        nL = len(labels) # num of labels
        
        # convert xyxy to xywh
        labels[:, 1:5] = xyxy2xywh(labels[:, 1:5]) / self.img_size
            
        labels_out = torch.zeros((nL, 6))
        labels_out[:, 1:] = torch.from_numpy(labels)
        
        # Normalize
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img, dtype=np.float32)  # uint8 to float32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0

        return torch.from_numpy(img), labels_out, img_path, (h, w)
    
def letterbox(img, height=416, color=(127.5, 127.5, 127.5)):
    # Resize a rectangular image to a padded square
    shape = img.shape[:2]  # shape = [height, width]
    ratio = float(height) / max(shape)  # ratio  = old / new
    new_shape = (round(shape[1] * ratio), round(shape[0] * ratio))
    dw = (height - new_shape[0]) / 2  # width padding
    dh = (height - new_shape[1]) / 2  # height padding
    top, bottom = round(dh - 0.1), round(dh + 0.1)
    left, right = round(dw - 0.1), round(dw + 0.1)
    img = cv2.resize(img, new_shape, interpolation=cv2.INTER_AREA)  # resized, no border
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # padded square
    return img, ratio, dw, dh



In [31]:
try:
    with open('a.txt','r') as f:
        d = f.read().splitlines()
except:
    print('no such files')
d

['1 2 3 4 5', '2 3 5 4 6', '3 6 5 8 9']

In [33]:
x = np.array([x.split() for x in d], dtype=np.float32)

In [34]:
x

array([[1., 2., 3., 4., 5.],
       [2., 3., 5., 4., 6.],
       [3., 6., 5., 8., 9.]], dtype=float32)